In [1]:
pip install libfmp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wfdb
from wfdb.plot import plot
import matplotlib.pyplot as plt
import cv2
from glob import glob
import numpy as np
from scipy import signal
import libfmp.c2
import librosa
from skimage.restoration import denoise_wavelet
directory="D:/ProyectoECG\mit-bih"
print("Working")

num_images=300

   




Working


In [3]:
def get_records():
    """ Obtener los paths para los archivos"""
    
    # Son 3 tipos de archivo
    paths = glob(directory+"/*.atr")

    #quita la extensión
    paths = [path[:-4] for path in paths]
    paths.sort()
    #print("I set paths")
    
    paths.remove('D:/ProyectoECG\mit-bih\\102-0')
    #print(paths)
    return paths

def beat_annotations(annotation, tipo):
    
    
    good = [tipo]   
    ids = np.in1d(annotation.symbol, good)

    # Posiciones de lo que contengan good
    beats = annotation.sample[ids]

    return beats
  
def segmentation(records,tipo):
    Normal = []
    for e in records:
        signals, fields = wfdb.rdsamp(e, channels = [0]) 
        #print("Segmentando")
        ann = wfdb.rdann(e, 'atr')
        good = [tipo]
        ids = np.in1d(ann.symbol, good)
        imp_beats = ann.sample[ids]
        beats = (ann.sample)
        for i in imp_beats:
            beats = list(beats)
            j = beats.index(i)
            if(j!=0 and j!=(len(beats)-1)):
                x = beats[j-1]
                y = beats[j+1]
                diff1 = abs(x - beats[j])//2
                diff2 = abs(y - beats[j])//2
                Normal.append(signals[beats[j] - diff1: beats[j] + diff2, 0])
    return Normal

In [4]:
def get_images(segmentos,inicial,path_):
    #print("Worked?")
    acum=0
    for count, i in enumerate(segmentos):
        #print("Dentro")
        
        """Wavelet denoise iniciando"""
        segmentos[2]=denoise_wavelet(segmentos[2],method='BayesShrink',mode='soft',wavelet_levels=3,wavelet='sym8',rescale_sigma='True')
        """Wavelet acabando"""
        plt.ioff()
        fig = plt.figure(frameon=False)
        #Applying Short-Time Fourier Transform
        f, t, Zxx = signal.stft(i,fs=360,nperseg=10)  
        plt.margins(x = 0)    
        #plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
        plt.pcolormesh(t, f, np.abs(Zxx))
        plt.xticks([]), plt.yticks([])
        for spine in plt.gca().spines.values():
            spine.set_visible(False)


        filename = directory + path_ + str(count+inicial)+'.png'
        fig.savefig(filename,bbox_inches='tight',pad_inches = 0)
        filename = './mit-bih' +path_+ str(count+inicial)+'.png'
        
        #print("Este es el path de la img",filename)
        #im_gray = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
        im_gray = cv2.imread(filename)
        #print("Estes es el shape de la img",im_gray.shape)
        im_gray = cv2.resize(im_gray, (256, 256), interpolation = cv2.INTER_LANCZOS4)
        
        cv2.imwrite(filename, im_gray)
        plt.close()
        #print("Fin")
        acum+=1
        if acum%num_images==0: #Only testing with 5 of each class
            break
    return inicial+acum

In [5]:
#Leyendo imagen

import cv2


#Le das la imagen, el filename="100.png", path_="/mit-imagenes/PAV/"
def cropping(image, filename,path_):
    #path_=path_+"/"
    #path_ = './mit-bih' +path_+ 'cropping/' +filename[:-4]
    """path_= ./mit-bih/mit-imagenes/PAV/cropping/ """
    path_=path_+filename[:-4]
    ####################
    #TOP
    #Izquierda
    crop = image[:200, :200]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'leftTop' + '.png', crop)
    
    #Centro
    crop = image[:200, 28:228]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'centerTop' + '.png', crop)
    
    #Derecha
    crop = image[:200, 56:]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'rightTop' + '.png', crop)
    
    ####################
    #Centro
    #Izquierda
    crop = image[28:228, :200]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'leftCenter' + '.png', crop)
    
    #Centro
    crop = image[28:228, 28:228]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'centerCenter' + '.png', crop)
    
    #Derecha
    crop = image[28:228, 56:]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'rightCenter' + '.png', crop)
    

    ####################
    #Inferior
    #Izquierda

    crop = image[56:, :192]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'leftBottom' + '.png', crop)
    
    #Centro
    crop = image[56:, 28:228]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'centerBottom' + '.png', crop)
    
    #Derecha
    crop = image[56:, 56:]
    crop = cv2.resize(crop, (256, 256))
    cv2.imwrite(path_ + 'rightBottom' + '.png', crop)



In [6]:
############################################
rec_=get_records()          #Obtener los paths de los records
#beats_ann=beat_annotations()

"""
PVC=V       premature ventricular contraction beat
PAB=/       paced beat
RBB=R       right bundle branch block beat
LBB=L       left bundle branch block beat
APC=A       atrial premature contraction beat
VFW=!       ventricular flutter wave
VEB=E       ventricular escape beat
N=N         normal

https://physionet.org/physiobank/database/html/mitdbdir/intro.htm#symbols"""
def returnPath(ch):
    path="/mit-imagenes/Tecnica2/"
    agregar=""
    if ch == 'V':
        agregar="PVC"
    elif ch == '/':
        agregar="PAB"
    elif ch == 'R':
        agregar="RBB"
    elif ch == 'L':
        agregar="LBB"
    elif ch == 'A':
        agregar="APC"
    elif ch == '!':
        agregar="VFW"
    elif ch == 'E':
        agregar="VEB"
    elif ch == 'N':
        agregar="N"
    path=path+agregar+'/'
    """se retornaría el path="/mit-imagenes/PVC/" """
    return path
#tipos=['N','L','R','A','V','/','E']

tipos=['V','/','R','L','A','!','E','N']

# fs=10e3
#    f, t, Zxx = signal.stft(segmentos, fs, nperseg=1000)
#    print(f.shape)

# tipoPath=returnPath(0)
# segmentos=segmentation(rec_,0)
# contador=get_images(segmentos,0,tipoPath)

contador=0
for i in tipos:
   tipoPath=returnPath(i)
   segmentos=segmentation(rec_,i)
   print(np.shape(segmentos))
   contador=get_images(segmentos,contador,tipoPath)

C:\Users\wilbe\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\wilbe\AppData\Local\Temp/ipykernel_2492/4154435417.py:16: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(t, f, np.abs(Zxx))


(7130,)
(7024,)
(7256,)
(8072,)
(2544,)
(472,)
(106,)
(75016,)


In [7]:

#RECORTANDO IMÁGENES
import os
#Obteniendo las imágenes de un folder path_="./mit-bih/mit-imagenes/PAB/"
#images to crop
images_to_crop=num_images

def cropping_images_folder(path_):
    images = []
    path_='./mit-bih'+path_
    final_path_=path_+'cropping/'     #
    count=0
    print(path_)
    print(final_path_)
    for filename in os.listdir(path_):
        #print(os.path.join(path_,filename))
        print("The filename is",filename)
        img = cv2.imread(os.path.join(path_,filename))
        if img is not None:
            print("is image")
        #     images.append(img)
            cropping(img,filename,final_path_)
            count+=1
        
        if count%images_to_crop==0:
            break
    return images


tipos_crop=['V','/','R','L','A','!','E']
#Recortando las imágenes de todos los folders excepto N
for i in tipos_crop:
   tipoPath=returnPath(i)
   cropping_images_folder(tipoPath)


""" PROBANDO Cropping
tipoPath=returnPath(tipos[1])
#pass /mit-imagenes/PAB/
#print(tipoPath)
cropping_images_folder(tipoPath)"""


./mit-bih/mit-imagenes/Tecnica2/PVC/
./mit-bih/mit-imagenes/Tecnica2/PVC/cropping/
The filename is 0.png
is image
The filename is 1.png
is image
The filename is 10.png
is image
The filename is 100.png
is image
The filename is 101.png
is image
The filename is 102.png
is image
The filename is 103.png
is image
The filename is 104.png
is image
The filename is 105.png
is image
The filename is 106.png
is image
The filename is 107.png
is image
The filename is 108.png
is image
The filename is 109.png
is image
The filename is 11.png
is image
The filename is 110.png
is image
The filename is 111.png
is image
The filename is 112.png
is image
The filename is 113.png
is image
The filename is 114.png
is image
The filename is 115.png
is image
The filename is 116.png
is image
The filename is 117.png
is image
The filename is 118.png
is image
The filename is 119.png
is image
The filename is 12.png
is image
The filename is 120.png
is image
The filename is 121.png
is image
The filename is 122.png
is image


' PROBANDO Cropping\ntipoPath=returnPath(tipos[1])\n#pass /mit-imagenes/PAB/\n#print(tipoPath)\ncropping_images_folder(tipoPath)'